In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 
%matplotlib inline
from sklearn.feature_extraction import text as TEXT
import re
from nltk.tokenize import RegexpTokenizer,wordpunct_tokenize, sent_tokenize
import os
from gensim.models import ldamulticore
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from nltk.stem import LancasterStemmer, PorterStemmer
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import nltk
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\varru\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Work cells

In [89]:
df_xls=pd.read_excel('C:/Users/varru/Documents/Topic_model_vj/chat_log.xlsx', sheetname="Chat_csv",usecols=['Message','SessionID','Origin'])


In [90]:
df_xls.head()

,SessionID,Origin,Message
0,144209,SYSTEM,{H Eugene Talbott} [JOINED SESSION]
1,144209,SYSTEM,[Virgilio Dela Pena] ASSIGNED SESSION
2,144209,SYSTEM,NEW SESSION CREATED
3,144209,H Eugene Talbott ...,H&I
4,144209,SYSTEM,[Virgilio Dela Pena] ACCEPTED SESSION


In [91]:
df_xls.dropna(inplace=True)

In [92]:
df_xls["Message"]=df_xls['Message'].astype(str)

In [93]:
df_xls=df_xls.groupby(["SessionID"]).sum()["Message"]

In [94]:
df_xls.head(4)

SessionID
144209    {H Eugene Talbott} [JOINED SESSION][Virgilio D...
144213    [Marilyn Lista] ASSIGNED SESSION{Selim Ataktur...
144215    [Virgilio Dela Pena] ASSIGNED SESSION{Jon J. D...
144218    [Virgilio Dela Pena] ASSIGNED SESSION{Valerie ...
Name: Message, dtype: object

In [96]:
document=[]
for i in range(len(df_xls)):
    document.append(str(df_xls.iloc[i]))
    

In [99]:
document[2]

'[Virgilio Dela Pena] ASSIGNED SESSION{Jon J. Danzak} [JOINED SESSION]NEW SESSION CREATED[Virgilio Dela Pena] ACCEPTED SESSION[Virgilio Dela Pena] JOINED SESSIONAEThank you for chatting with us today. How can I help you?I made an error when submitting my annual physical form.  I accidentally checked "tobacco user".   I submitted a corrected form some time ago (at least a month).  I see that I still have a tobcco surcharge on my 2018 benefits.How can I get that fixed?I\'ll be happy to assist you with that. It will be just a minute while I access your records.harges will be automatically removed and you will not be responsible for payment.I apologize for this discrepancy,Mr. Danzak,  Given when you said that data was faxed over, it is possible that it has simply not processed in our system yet as this can often take up to three weeks. Once the information is processed, the appropriate surcOk, I\'ll wait another 2 weeks check again.   It may be on the \'cusp\' of being entered..updated et

In [100]:
lemmer=WordNetLemmatizer()
stemmer = PorterStemmer()
regex=RegexpTokenizer("\\b[a-zA-Z][a-zA-Z][a-zA-Z]+\\b")

def stem_tokens(tokens, stemmer):
    stemmed = []
    
    for item in tokens:
        #stemmed.append(stemmer.stem(item))
        stemmed.append(lemmer.lemmatize(item,"v"))
    return stemmed

def tokenize(text):
    text=text.lower()
    text=re.sub("[\(\[].*?[\)\]]", "", text)
    
    tokens = regex.tokenize(text)
   
    stems = stem_tokens(tokens, stemmer)
    return stems

stop=['virgilio', 'dela', 'pena','thank', 'how', 'good', 'morning','marilyn',
      'lista', 'selim', 'atakturk', 'samantha', 'wagner','this', 'can', 'okay',
      'miss', 'spallo', 'would', 'so', 'cigna', 'yes', 'the','eugene', 'talbott', 'join', 'session', 'virgilio',
      'assign', 'new', 'create', 'accept', 'chat','jhoana','cruz',"umali","andrew","klaudine","bettina","ocampo","carla"
      ,"alejandrino","cresencia","bustos","allan",'website','contact','records''online','record','waiting','web',
             'new','click','site','since','thanks','happy','david',
             'already','year','like','send','yes','last','right','also'
             ,'give','know','would','get','make','wonderful','phone',
             'want','something','heard','see','would','may','else',
             'one','day','minute','see word','today','inform','using',
             'call','good','morninig','anything','need','chat','thank','calling','please','moment',
             'check','welcome','nan','assigned','session','can','help','still','session','assist','inform',
             'number','checking','chatting','access','time','change','acknowledg',"customer",
              "hello","just","minute","send","use","try","sessionsession","sessionend",""]
stop_wd=TEXT.ENGLISH_STOP_WORDS.union(stop)

# Create a CountVectorizer for parsing/counting words

In [108]:
tokenize(document[1])

['assign',
 'session',
 'selim',
 'atakturk',
 'new',
 'session',
 'create',
 'accept',
 'session',
 'join',
 'sessionaethank',
 'you',
 'for',
 'chat',
 'with',
 'today',
 'how',
 'can',
 'help',
 'you',
 'see',
 'that',
 'tobacco',
 'surcharge',
 'still',
 'plan',
 'make',
 'the',
 'necessary',
 'call',
 'and',
 'the',
 'talk',
 'for',
 'that',
 'why',
 'ithat',
 'surcharge',
 'there',
 'apologize',
 'for',
 'that',
 'inconvenience',
 'let',
 'assist',
 'you',
 'that',
 'concern',
 'let',
 'just',
 'pull',
 'your',
 'account',
 'one',
 'moment',
 'please',
 'here',
 'wait',
 'thank',
 'you',
 'for',
 'patiently',
 'wait',
 'still',
 'verify',
 'the',
 'completion',
 'your',
 'screen',
 'please',
 'stay',
 'online',
 'waitingthank',
 'you',
 'thank',
 'you',
 'for',
 'patiently',
 'wait',
 'may',
 'ask',
 'you',
 'be',
 'tobacco',
 'smoker',
 'yesi',
 'see',
 'be',
 'you',
 'able',
 'the',
 'cessation',
 'program',
 'be',
 'require',
 'call',
 'them',
 'which',
 'do',
 'they',
 'give'

In [101]:
tf=TfidfVectorizer(stop_words=stop_wd,ngram_range=(1,2),tokenizer=tokenize
                   ,max_df=0.7)                   
                 
mat_cv=tf.fit_transform(iter(document))


In [ ]:
#count_vectorizer = CountVectorizer(tokenizer=tokenize,stop_words=stop_wd,min_df=10,max_df=0.7,)
#mat_cv=count_vectorizer.fit_transform(iter(document))


In [102]:
#len(count_vectorizer.get_feature_names())
len(tf.get_feature_names())

105858

In [105]:
#count_vectorizer.get_feature_names()

In [ ]:
#cv_vecs=pd.DataFrame(mat_cv.todense(), columns=count_vectorizer.get_feature_names()).head()
#cv_vecs=pd.DataFrame(mat_cv.todense(), columns=tf.get_feature_names()).head()


# Using LDA Model

In [103]:
# Convert sparse matrix of counts to a gensim corpus for LDA Model

cv_2corpus = matutils.Sparse2Corpus(mat_cv.transpose())
ld_words2idf=dict((v,k) for k,v in tf.vocabulary_.items())
ld_words2idf=corpora.Dictionary.from_corpus(corpus=cv_2corpus,id2word=ld_words2idf)



2019-04-29 15:47:04,375 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-04-29 15:47:04,704 : INFO : built Dictionary(105858 unique tokens: ['sessionnew', 'createdh', 'sessionthank', 'case', 'open']...) from 4369 documents (total 31135 corpus positions)


In [106]:
import multiprocessing

multiprocessing.cpu_count()

8

In [107]:
# Using LDA Model
logging.basicConfig(filename='~/gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

lda=ldamulticore.LdaMulticore(cv_2corpus,id2word=ld_words2idf,num_topics=10,passes=50,eval_every=None,workers=7,
                              chunksize=10000)

2019-04-29 15:47:42,589 : INFO : using symmetric alpha at 0.1
2019-04-29 15:47:42,591 : INFO : using symmetric eta at 0.1
2019-04-29 15:47:42,606 : INFO : using serial LDA version on this node
2019-04-29 15:47:46,377 : INFO : running online LDA training, 10 topics, 50 passes over the supplied corpus of 4369 documents, updating every 70000 documents, evaluating every ~0 documents, iterating 50x with a convergence threshold of 0.001000
2019-04-29 15:47:46,387 : INFO : training LDA model using 7 processes
2019-04-29 15:47:46,607 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #4369/4369, outstanding queue size 1
2019-04-29 15:47:54,745 : INFO : topic #6 (0.100): 0.001*"sessioncustomer" + 0.001*"sessiontimed" + 0.001*"insurance" + 0.001*"cancel" + 0.001*"card" + 0.001*"benefit" + 0.001*"end leave" + 0.000*"plan" + 0.000*"cancel sessionsession" + 0.000*"sessioncustomer cancel"
2019-04-29 15:47:54,747 : INFO : topic #5 (0.100): 0.004*"sessiontimed" + 0.003*"sessiontimed sess

2019-04-29 15:48:14,661 : INFO : topic diff=0.009372, rho=0.428869
2019-04-29 15:48:14,735 : INFO : PROGRESS: pass 5, dispatched chunk #0 = documents up to #4369/4369, outstanding queue size 1
2019-04-29 15:48:19,532 : INFO : topic #4 (0.100): 0.001*"benefit" + 0.001*"plan" + 0.001*"dental" + 0.001*"account" + 0.001*"coverage" + 0.001*"option" + 0.001*"end leave" + 0.001*"insurance" + 0.001*"center" + 0.001*"card"
2019-04-29 15:48:19,535 : INFO : topic #7 (0.100): 0.009*"sessiontimed" + 0.008*"sessiontimed sessiontimed" + 0.005*"sessioncustomer" + 0.003*"sessioncustomer leave" + 0.002*"cancel sessionsession" + 0.002*"sessioncustomer cancel" + 0.002*"leave sessioncustomer" + 0.002*"cancel" + 0.002*"sessiontimed sessioncustomer" + 0.002*"createdtimed"
2019-04-29 15:48:19,537 : INFO : topic #3 (0.100): 0.001*"sessioncustomer" + 0.001*"sessiontimed" + 0.001*"benefit" + 0.001*"cancel" + 0.001*"cancel sessionsession" + 0.001*"sessioncustomer cancel" + 0.001*"leave sessioncustomer" + 0.001*"a

KeyboardInterrupt: 

In [12]:
lda.print_topics(num_words=15,num_topics=10)

2019-04-29 12:53:13,080 : INFO : topic #0 (0.100): 0.431*"leave" + 0.006*"nope" + 0.002*"term" + 0.002*"disability" + 0.002*"term disability" + 0.002*"online" + 0.001*"blue" + 0.001*"short" + 0.001*"log" + 0.001*"pin" + 0.001*"long term" + 0.001*"john" + 0.001*"cross" + 0.001*"blue cross" + 0.001*"short term"
2019-04-29 12:53:13,083 : INFO : topic #1 (0.100): 0.020*"insurance" + 0.012*"health" + 0.010*"coverage" + 0.009*"health insurance" + 0.008*"card" + 0.006*"don" + 0.005*"email" + 0.005*"cover" + 0.005*"plan" + 0.004*"tab" + 0.004*"push" + 0.004*"email push" + 0.004*"medical" + 0.004*"insurance card" + 0.004*"life"
2019-04-29 12:53:13,085 : INFO : topic #2 (0.100): 0.013*"account" + 0.013*"correct" + 0.011*"payment" + 0.006*"benefit" + 0.006*"question" + 0.005*"pull" + 0.005*"receive" + 0.004*"permission" + 0.004*"online" + 0.004*"look" + 0.004*"pull account" + 0.004*"permission browse" + 0.004*"days" + 0.004*"browse" + 0.004*"mail"
2019-04-29 12:53:13,087 : INFO : topic #3 (0.100)

[(0,
  '0.431*"leave" + 0.006*"nope" + 0.002*"term" + 0.002*"disability" + 0.002*"term disability" + 0.002*"online" + 0.001*"blue" + 0.001*"short" + 0.001*"log" + 0.001*"pin" + 0.001*"long term" + 0.001*"john" + 0.001*"cross" + 0.001*"blue cross" + 0.001*"short term"'),
 (1,
  '0.020*"insurance" + 0.012*"health" + 0.010*"coverage" + 0.009*"health insurance" + 0.008*"card" + 0.006*"don" + 0.005*"email" + 0.005*"cover" + 0.005*"plan" + 0.004*"tab" + 0.004*"push" + 0.004*"email push" + 0.004*"medical" + 0.004*"insurance card" + 0.004*"life"'),
 (2,
  '0.013*"account" + 0.013*"correct" + 0.011*"payment" + 0.006*"benefit" + 0.006*"question" + 0.005*"pull" + 0.005*"receive" + 0.004*"permission" + 0.004*"online" + 0.004*"look" + 0.004*"pull account" + 0.004*"permission browse" + 0.004*"days" + 0.004*"browse" + 0.004*"mail"'),
 (3,
  '0.039*"cancel" + 0.016*"option" + 0.009*"event" + 0.009*"life" + 0.009*"reach" + 0.008*"reach option" + 0.008*"continue" + 0.008*"life event" + 0.007*"respond" +

In [15]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[cv_2corpus]
pd.DataFrame(lda_corpus[1])


,0,1
0,0,0.1
1,1,0.1
2,2,0.1
3,3,0.1
4,4,0.1
5,5,0.1
6,6,0.1
7,7,0.1
8,8,0.1
9,9,0.1


In [17]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

In [18]:
print("Topic Values", lda_docs[0]) # print Topic Values for document 0 

Topic Values [(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.1), (6, 0.1), (7, 0.1), (8, 0.1), (9, 0.1)]


In [19]:
lda.num_topics

10

In [20]:

#consolidated data frame with document number , topic_number, and highest topic score and related topic terms
number_of_topics=10
topics=lda.show_topics(num_topics=number_of_topics)
topic_df=pd.DataFrame.from_records(topics,columns=["topic_id","topic_terms"])

highest_score_topic_lda_docs=[sorted(i,key=lambda x: x[1],reverse=True)[0] for i in lda_docs]

topics_by_documnent_numbers=pd.DataFrame(data=highest_score_topic_lda_docs,columns=["topic_number","topic_score"])

topics_by_documnent_numbers=topics_by_documnent_numbers.join(topic_df,on='topic_number')
topics_by_documnent_numbers.drop("topic_id",axis=1,inplace=True)

docnument_number=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers=topics_by_documnent_numbers.reset_index()

topics_by_documnent_numbers.rename(columns={"index":"document_number"},inplace=True)

In [21]:
topics_by_documnent_numbers.head(10)
topics_by_documnent_numbers.topic_number.value_counts()

0    58517
5     9710
2     5826
4     5789
1     5424
9     5239
3     5122
8     4724
6     4171
7     2995
Name: topic_number, dtype: int64

In [ ]:
#lda.get_document_topics()
#lda.get_document_topics(ld_words2idf)

In [17]:
import seaborn as sns
docs_per_topic=(topics_by_documnent_numbers)

In [23]:
#docs_per_topic.to_csv("docs_per_topic.csv")

In [26]:
#prints text and topic_terms
def get_document2topic(doc_number):
    
    return(topics_by_documnent_numbers["topic_terms"].iloc[doc_number],document[doc_number])

topic, text=get_document2topic(8)
print("\n"+"\033[1m")
print("Topic_terms:" ,(tokenize(topic)),"\n")
print("Document:",text,"...")



Topic_terms: ['information', 'benefit', 'center', 'unresponsive', 'think', 'service', 'benefit', 'service', 'service', 'center', 'wait', 'benefit', 'center'] 

Document: And am checking in on the case to see what the disposition is ...


In [25]:
# Consolidated dictionary of topic numbers to text

mapped_dictionary_document={}

for i in range(len(topics_by_documnent_numbers)):
    mapped_dictionary_document[i]=get_document2topic(i)

In [ ]:
#mapped_dictionary_document.values()

## Visualization

In [ ]:

import pyLDAvis.gensim

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, cv_2corpus,ld_words2idf )

In [ ]:
top_topics = lda.top_topics(cv_2corpus, topn=20)